In [1]:
import os
import pandas as pd
import numpy as np
import re
import datetime as dt
import matplotlib.pyplot as plt

In [2]:
#Importing the dataframe from attacks.csv

df = pd.read_csv('input/attacks.csv', encoding = 'latin-1') 

In [3]:
#Displaying the dataframe
display(df)

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
0,2018.06.25,25-Jun-2018,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,...,White shark,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.25,2018.06.25,6303.0,NaN,NaN
1,2018.06.18,18-Jun-2018,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,...,NaN,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.18,2018.06.18,6302.0,NaN,NaN
2,2018.06.09,09-Jun-2018,2018.0,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,...,NaN,"K.McMurray, TrackingSharks.com",2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.09,2018.06.09,6301.0,NaN,NaN
3,2018.06.08,08-Jun-2018,2018.0,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,...,2 m shark,"B. Myatt, GSAF",2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.08,2018.06.08,6300.0,NaN,NaN
4,2018.06.04,04-Jun-2018,2018.0,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,...,"Tiger shark, 3m",A .Kipper,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.04,2018.06.04,6299.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25718,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25719,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25720,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25721,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
#Displaying the columns in the dataframe
df.columns

Index(['Case Number', 'Date', 'Year', 'Type', 'Country', 'Area', 'Location',
       'Activity', 'Name', 'Sex ', 'Age', 'Injury', 'Fatal (Y/N)', 'Time',
       'Species ', 'Investigator or Source', 'pdf', 'href formula', 'href',
       'Case Number.1', 'Case Number.2', 'original order', 'Unnamed: 22',
       'Unnamed: 23'],
      dtype='object')

In [5]:
#Renaming some columns to express their content
df.rename(columns ={"Case Number": "Case_Number",
                    "Fatal (Y/N)": "Fatal_(Y/N)",
                    "Species ":"Species",
                    "Investigator or Source":"Investigator_or_Source",
                    "href formula": "PDF_Link",
                    "href": "PDF_Link_1",
                    "Case Number.1":"Case_Number1",
                    "Case Number.2":"Case_Number_2",
                    "original order": "Original_Order"
                       } , inplace = True)

#Removing spaces from begining and end for every column
df.columns = df.columns.str.strip()

#Transforming None types into "Not Available" for every column
df.fillna("Not Available", inplace=True)

display(df.head())
display(df.tail())

,Case_Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator_or_Source,pdf,PDF_Link,PDF_Link_1,Case_Number1,Case_Number_2,Original_Order,Unnamed: 22,Unnamed: 23
0,2018.06.25,25-Jun-2018,2018,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,...,White shark,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.25,2018.06.25,6303,Not Available,Not Available
1,2018.06.18,18-Jun-2018,2018,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,...,Not Available,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.18,2018.06.18,6302,Not Available,Not Available
2,2018.06.09,09-Jun-2018,2018,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,...,Not Available,"K.McMurray, TrackingSharks.com",2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.09,2018.06.09,6301,Not Available,Not Available
3,2018.06.08,08-Jun-2018,2018,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,...,2 m shark,"B. Myatt, GSAF",2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.08,2018.06.08,6300,Not Available,Not Available
4,2018.06.04,04-Jun-2018,2018,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,...,"Tiger shark, 3m",A .Kipper,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.04,2018.06.04,6299,Not Available,Not Available


,Case_Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator_or_Source,pdf,PDF_Link,PDF_Link_1,Case_Number1,Case_Number_2,Original_Order,Unnamed: 22,Unnamed: 23
25718,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,...,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available
25719,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,...,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available
25720,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,...,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available
25721,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,...,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available
25722,xx,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,...,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available


The hypothesis to be refuted is the following: The country in the world with the most deaths of men, who were surfing, due to a white shark attack from XX century is Australia. 

En primer lugar, tras analizar el fichero encontramos que algunas columnas no cuentan con datos suficientes para llevar a cabo una análisis. No se utiliza la columna "Species" ya que el número de celdas vacías


We will make use of the following columns to be able to conclude:
- Year: > 1900
- Country: Australia
- Activity: Surfing
- Sex: M
- Fatal (Y/N): Y (deaths)



In [6]:
#Creating a new DF with the columns needed to validate the hypothesis

hypo = df[["Year", "Country", "Activity", "Sex", "Fatal_(Y/N)", "Species"]]
display(hypo.head())

,Year,Country,Activity,Sex,Fatal_(Y/N),Species
0,2018,USA,Paddling,F,N,White shark
1,2018,USA,Standing,F,N,Not Available
2,2018,USA,Surfing,M,N,Not Available
3,2018,AUSTRALIA,Surfing,M,N,2 m shark
4,2018,MEXICO,Free diving,M,N,"Tiger shark, 3m"


In [7]:
#We start by analyzing "Year" column in order to clean wrong values.
#Showing all values for this column. We can identify several uncorrect values.
pd.unique(hypo["Year"])


array([2018.0, 2017.0, 'Not Available', 2016.0, 2015.0, 2014.0, 2013.0,
       2012.0, 2011.0, 2010.0, 2009.0, 2008.0, 2007.0, 2006.0, 2005.0,
       2004.0, 2003.0, 2002.0, 2001.0, 2000.0, 1999.0, 1998.0, 1997.0,
       1996.0, 1995.0, 1984.0, 1994.0, 1993.0, 1992.0, 1991.0, 1990.0,
       1989.0, 1969.0, 1988.0, 1987.0, 1986.0, 1985.0, 1983.0, 1982.0,
       1981.0, 1980.0, 1979.0, 1978.0, 1977.0, 1976.0, 1975.0, 1974.0,
       1973.0, 1972.0, 1971.0, 1970.0, 1968.0, 1967.0, 1966.0, 1965.0,
       1964.0, 1963.0, 1962.0, 1961.0, 1960.0, 1959.0, 1958.0, 1957.0,
       1956.0, 1955.0, 1954.0, 1953.0, 1952.0, 1951.0, 1950.0, 1949.0,
       1948.0, 1848.0, 1947.0, 1946.0, 1945.0, 1944.0, 1943.0, 1942.0,
       1941.0, 1940.0, 1939.0, 1938.0, 1937.0, 1936.0, 1935.0, 1934.0,
       1933.0, 1932.0, 1931.0, 1930.0, 1929.0, 1928.0, 1927.0, 1926.0,
       1925.0, 1924.0, 1923.0, 1922.0, 1921.0, 1920.0, 1919.0, 1918.0,
       1917.0, 1916.0, 1915.0, 1914.0, 1913.0, 1912.0, 1911.0, 1910.0,
     

In [8]:
#Displaying this values on original DF to try to fix them
display(df[df.Year.isin([5, 77, 500])])
#Unexpectedly,this values are correct so we will focus on "0" cases. 
#As we can see, "Date" column also contains the year.We import this column into "hypo" 
hypo["Date"] = df["Date"]

,Case_Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator_or_Source,pdf,PDF_Link,PDF_Link_1,Case_Number1,Case_Number_2,Original_Order,Unnamed: 22,Unnamed: 23
6174,0500.00.00,Circa 500 A.D.,500,Unprovoked,MEXICO,Not Available,Not Available,Not Available,male,M,...,Not Available,J. Castro,500AD-Mexico.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,0500.00.00,0500.00.00,129,Not Available,Not Available
6175,0077.00.00,77 A.D.,77,Unprovoked,Not Available,Ionian Sea,Not Available,Sponge diving,males,M,...,Not Available,Perils mentioned by Pliny the Elder (23 A.D. t...,77AD-Pliny.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,0077.00.00,0077.00.00,128,Not Available,Not Available
6176,0005.00.00,Ca. 5 A.D.,5,Unprovoked,AUSTRALIA,New South Wales,Bondi,Not Available,male,M,...,Not Available,Waverly Library,0005.00.00-AustralianAboriginal.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,0005.00.00,0005.00.00,127,Not Available,Not Available


<ipython-input-8-b827c143931f>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hypo["Date"] = df["Date"]


In [9]:
#Appending a column "Date_match" with the years extracted from "Date"

x = []
for l in hypo["Date"]:
    x.append(str(l))

z =[]
for l in x:
    match = re.match(r'.*([1-3][0-9]{3})', l)
    if match is not None:
        z.append(match[1])
    else:
        z.append("no match")
hypo["Date_match"] = z
hypo.head()

<ipython-input-9-a3cdc2ff7549>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hypo["Date_match"] = z


,Year,Country,Activity,Sex,Fatal_(Y/N),Species,Date,Date_match
0,2018,USA,Paddling,F,N,White shark,25-Jun-2018,2018
1,2018,USA,Standing,F,N,Not Available,18-Jun-2018,2018
2,2018,USA,Surfing,M,N,Not Available,09-Jun-2018,2018
3,2018,AUSTRALIA,Surfing,M,N,2 m shark,08-Jun-2018,2018
4,2018,MEXICO,Free diving,M,N,"Tiger shark, 3m",04-Jun-2018,2018


In [10]:
#Not Available rows are empty and "no match" rows contain years before 1000 so we exclude them.
hypo["Year_Date"] = np.where(hypo["Year"] == 0, hypo["Date_match"], hypo["Year"])
hypo.groupby("Year_Date").count()



<ipython-input-10-fff5bd07ae18>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hypo["Year_Date"] = np.where(hypo["Year"] == 0, hypo["Date_match"], hypo["Year"])


,Year,Country,Activity,Sex,Fatal_(Y/N),Species,Date,Date_match
Year_Date,,,,,,,,
5.0,1,1,1,1,1,1,1,1
77.0,1,1,1,1,1,1,1,1
500.0,1,1,1,1,1,1,1,1
1543.0,1,1,1,1,1,1,1,1
1554.0,1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...
2011,2,2,2,2,2,2,2,2
2012,3,3,3,3,3,3,3,3
2016,1,1,1,1,1,1,1,1


In [11]:
#Deleting "Not Available" and "no match" rows
hypo = hypo.drop(hypo[(hypo['Year_Date']=='Not Available') | (hypo['Year_Date'] =='no match')].index)

pd.unique(hypo["Year_Date"])


array([2018.0, 2017.0, 2016.0, 2015.0, 2014.0, 2013.0, 2012.0, 2011.0,
       2010.0, 2009.0, 2008.0, 2007.0, 2006.0, 2005.0, 2004.0, 2003.0,
       2002.0, 2001.0, 2000.0, 1999.0, 1998.0, 1997.0, 1996.0, 1995.0,
       1984.0, 1994.0, 1993.0, 1992.0, 1991.0, 1990.0, 1989.0, 1969.0,
       1988.0, 1987.0, 1986.0, 1985.0, 1983.0, 1982.0, 1981.0, 1980.0,
       1979.0, 1978.0, 1977.0, 1976.0, 1975.0, 1974.0, 1973.0, 1972.0,
       1971.0, 1970.0, 1968.0, 1967.0, 1966.0, 1965.0, 1964.0, 1963.0,
       1962.0, 1961.0, 1960.0, 1959.0, 1958.0, 1957.0, 1956.0, 1955.0,
       1954.0, 1953.0, 1952.0, 1951.0, 1950.0, 1949.0, 1948.0, 1848.0,
       1947.0, 1946.0, 1945.0, 1944.0, 1943.0, 1942.0, 1941.0, 1940.0,
       1939.0, 1938.0, 1937.0, 1936.0, 1935.0, 1934.0, 1933.0, 1932.0,
       1931.0, 1930.0, 1929.0, 1928.0, 1927.0, 1926.0, 1925.0, 1924.0,
       1923.0, 1922.0, 1921.0, 1920.0, 1919.0, 1918.0, 1917.0, 1916.0,
       1915.0, 1914.0, 1913.0, 1912.0, 1911.0, 1910.0, 1909.0, 1908.0,
      

In [34]:
# Converting into int "Year_Date" column 
x = []
for h in hypo["Year_Date"]:
    x.append(int(h))
hypo["Year_Date"] = x

# Filtering from XXth century and removing "Year", "Date_match" and "Date"
hypo =hypo[hypo['Year_Date'] > 1900]
hypo = hypo[["Year_Date","Country", "Activity", "Sex", "Fatal_(Y/N)", "Species"]]
hypo

,Year_Date,Country,Activity,Sex,Fatal_(Y/N),Species
0,2018,USA,Paddling,F,N,White shark
1,2018,USA,Standing,F,N,Not Available
2,2018,USA,Surfing,M,N,Not Available
3,2018,AUSTRALIA,Surfing,M,N,2 m shark
4,2018,MEXICO,Free diving,M,N,"Tiger shark, 3m"
...,...,...,...,...,...,...
6295,1906,AUSTRALIA,Fishing,M,Y,Blue pointers
6296,1906,AUSTRALIA,Swimming,M,Y,Said to involve a grey nurse shark that leapt ...
6297,1903,AUSTRALIA,Diving,M,Y,Not Available
6298,1903,AUSTRALIA,Pearl diving,M,Y,Not Available


In [14]:
#Printing all values in "Country". This column contains some misstatements for some countries
print(pd.unique(hypo["Country"]))
#By printing thetop25 countries for attacks, we are proving that the impact of misstatements is residual.
hypo.groupby(by="Country").count().sort_values("Year_Date", ascending=False).head(25)

['USA' 'AUSTRALIA' 'MEXICO' 'BRAZIL' 'ENGLAND' 'SOUTH AFRICA' 'THAILAND'
 'COSTA RICA' 'MALDIVES' 'BAHAMAS' 'NEW CALEDONIA' 'ECUADOR' 'MALAYSIA'
 'LIBYA' 'Not Available' 'CUBA' 'MAURITIUS' 'NEW ZEALAND' 'SPAIN' 'SAMOA'
 'SOLOMON ISLANDS' 'JAPAN' 'EGYPT' 'ST HELENA, British overseas territory'
 'COMOROS' 'REUNION' 'FRENCH POLYNESIA' 'UNITED KINGDOM'
 'UNITED ARAB EMIRATES' 'PHILIPPINES' 'INDONESIA' 'CHINA' 'COLUMBIA'
 'CAPE VERDE' 'Fiji' 'DOMINICAN REPUBLIC' 'CAYMAN ISLANDS' 'ARUBA'
 'MOZAMBIQUE' 'FIJI' 'PUERTO RICO' 'ITALY' 'ATLANTIC OCEAN' 'GREECE'
 'ST. MARTIN' 'FRANCE' 'PAPUA NEW GUINEA' 'TRINIDAD & TOBAGO' 'KIRIBATI'
 'ISRAEL' 'DIEGO GARCIA' 'TAIWAN' 'JAMAICA' 'PALESTINIAN TERRITORIES'
 'GUAM' 'SEYCHELLES' 'BELIZE' 'NIGERIA' 'TONGA' 'SCOTLAND' 'CANADA'
 'CROATIA' 'SAUDI ARABIA' 'CHILE' 'ANTIGUA' 'KENYA' 'RUSSIA'
 'TURKS & CAICOS' 'UNITED ARAB EMIRATES (UAE)' 'AZORES' 'SOUTH KOREA'
 'MALTA' 'VIETNAM' 'MADAGASCAR' 'PANAMA' 'SOMALIA' 'NEVIS'
 'BRITISH VIRGIN ISLANDS' 'NORWAY' 'SENEGAL

,Year_Date,Activity,Sex,Fatal_(Y/N),Species
Country,,,,,
USA,2092,2092,2092,2092,2092
AUSTRALIA,1190,1190,1190,1190,1190
SOUTH AFRICA,552,552,552,552,552
PAPUA NEW GUINEA,132,132,132,132,132
BRAZIL,108,108,108,108,108
BAHAMAS,103,103,103,103,103
NEW ZEALAND,98,98,98,98,98
MEXICO,78,78,78,78,78
ITALY,62,62,62,62,62


In [15]:
pd.unique(hypo["Activity"])

array(['Paddling', 'Standing', 'Surfing', ...,
       'Crew swimming alongside their anchored ship',
       '4 men were bathing', 'Wreck of  large double sailing canoe'],
      dtype=object)

In [52]:
#Extrating Surf, Swimming, fishing an diving from "Activity" and creating a new column "Activity_new"
x= []

for h in hypo["Activity"]:
    y =(re.search(r"swim|Swim|fish|Fish|div|Div|surf|Surf|", h))
    x.append(y.group().lower())

#Filtering Surf, Swimming, fishing an diving        
hypo["Activity_new"] = x     
hypo =hypo[(hypo.Activity_new == "surf")|
      (hypo.Activity_new == "fish")|
      (hypo.Activity_new == "div")|
      (hypo.Activity_new == "swim")]

hypo.groupby("Activity_new").count()
                                                                                

,Year_Date,Country,Activity,Sex,Fatal_(Y/N),Species
Activity_new,,,,,,
div,213,213,213,213,213,213
fish,559,559,559,559,559,559
surf,1061,1061,1061,1061,1061,1061
swim,941,941,941,941,941,941


In [60]:
#Exploring "Fatal_(Y/N)" column
hypo.groupby("Fatal_(Y/N)").count()


#Deleting spaces 
hypo["Fatal_(Y/N)"] = hypo["Fatal_(Y/N)"].str.strip()

hypo.groupby("Fatal_(Y/N)").count()

,Year_Date,Country,Activity,Sex,Species,Activity_new
Fatal_(Y/N),,,,,,
N,2099,2099,2099,2099,2099,2099
Not Available,177,177,177,177,177,177
UNKNOWN,16,16,16,16,16,16
Y,482,482,482,482,482,482


In [62]:
#hypo.groupby(by="Country").count().sort_values("Year_Date", ascending = False)
hypo[(hypo['Country'] == 'AUSTRALIA')]

,Year_Date,Country,Activity,Sex,Fatal_(Y/N),Species,Activity_new
3,2018,AUSTRALIA,Surfing,M,N,2 m shark,surf
16,2018,AUSTRALIA,Surfing,M,N,Shark involvement not confirmed,surf
19,2018,AUSTRALIA,Surfing,M,N,Not Available,surf
20,2018,AUSTRALIA,Surfing,M,N,Questionable,surf
21,2018,AUSTRALIA,Surfing,M,N,3 m shark,surf
...,...,...,...,...,...,...,...
6293,1906,AUSTRALIA,Fishing,M,Y,Blue pointer,fish
6294,1906,AUSTRALIA,Fishing,M,Y,Blue pointer,fish
6295,1906,AUSTRALIA,Fishing,M,Y,Blue pointers,fish
6296,1906,AUSTRALIA,Swimming,M,Y,Said to involve a grey nurse shark that leapt ...,swim


In [63]:
attacks = hypo.groupby(by="Country").count().sort_values("Year_Date", ascending = False).head(10)
attacks = attacks.reset_index()
attacks

,Country,Year_Date,Activity,Sex,Fatal_(Y/N),Species,Activity_new
0,USA,1133,1133,1133,1133,1133,1133
1,AUSTRALIA,604,604,604,604,604,604
2,SOUTH AFRICA,276,276,276,276,276,276
3,BRAZIL,76,76,76,76,76,76
4,PAPUA NEW GUINEA,51,51,51,51,51,51
5,NEW ZEALAND,50,50,50,50,50,50
6,MEXICO,36,36,36,36,36,36
7,REUNION,29,29,29,29,29,29
8,MOZAMBIQUE,29,29,29,29,29,29
9,ITALY,29,29,29,29,29,29


In [64]:
deaths = hypo[(hypo['Fatal_(Y/N)'] == 'Y')]

deaths = deaths.groupby(by="Country").count().sort_values("Year_Date", ascending = False)
deaths = deaths.reset_index()
deaths

,Country,Year_Date,Activity,Sex,Fatal_(Y/N),Species,Activity_new
0,AUSTRALIA,107,107,107,107,107,107
1,USA,65,65,65,65,65,65
2,SOUTH AFRICA,48,48,48,48,48,48
3,BRAZIL,26,26,26,26,26,26
4,PAPUA NEW GUINEA,23,23,23,23,23,23
...,...,...,...,...,...,...,...
69,REUNION ISLAND,1,1,1,1,1,1
70,SAMOA,1,1,1,1,1,1
71,SAUDI ARABIA,1,1,1,1,1,1
72,SENEGAL,1,1,1,1,1,1


In [65]:
country_attack = attacks['Country'].head(5).tolist()
attack = attacks['Activity'].head(5).tolist()

country_death = deaths['Country'].head(5).tolist()
death = deaths['Activity'].head(5).tolist()

fig.autofmt_xdate

plt.bar(country_death, death, align = "center", color="blue", width = 0.5)
plt.bar(country_attack, attack, align = "center",color="orange",width = 0.2)

plt.title("DEATHS BY COUNTRY")
fig.autofmt_xdate
plt.show()


NameError: name 'fig' is not defined

In [66]:
country_attack = attacks['Country'].head(5).tolist()
attack = attacks['Activity'].head(5).tolist()